In [1]:
import pandas as pd

In [13]:
!

 alec_monday_march13.ipynb
 basic_box_team_games_preproc.ipynb
'Combine preprocessed data pickles.ipynb'
 elliot.ipynb
 friday_pitch.ipynb
 hyperopt_gbdt_players.ipynb
 model_julia.ipynb
 nba_api_example-Copy1.ipynb
 nba_api_example.ipynb
 nba_api_v2.ipynb
 neural_net.ipynb
 NGBoost_julia.ipynb
 Preproc_Alec_Updated.ipynb
 preprocessing_alec.ipynb
 preprocessing_games_df.ipynb
 preprocessing_julia2.ipynb
 preprocessing_julia.ipynb
 production_test_julia.ipynb
 pysbr_sourcing-alec.ipynb
 pysbr_sourcing.ipynb
 pysbrtutorial.ipynb
 Ricky_advanced_box_baseline_model.ipynb
 Ricky_basic_baseline_model.ipynb
 Ricky_model.ipynb
 Ricky_player_advanced_boxscore.ipynb
 Ricky_XGboost.ipynb
 sbr_bet_future_scraping_julia.ipynb
 sbr_bet_history_scraping.ipynb
 secondary_model_alec-Copy1.ipynb
 secondary_model_alec.ipynb
 Untitled.ipynb
 XGboost_Alec.ipynb


In [23]:
!pwd

/home/alec/code/alecmatt5/nba_betting_analysis/nba_betting_analysis/backend/notebooks


In [33]:
import os
os.chdir('/home/alec/code/alecmatt5/nba_betting_analysis/nba_betting_analysis/backend/py_files')
from preprocess import get_basic_boxscores

In [34]:
os.chdir('/home/alec/code/alecmatt5/nba_betting_analysis/nba_betting_analysis/backend/notebooks')
from bs4 import BeautifulSoup
import requests
import time

In [36]:
!pwd

/home/alec/code/alecmatt5/nba_betting_analysis/nba_betting_analysis/backend/notebooks


In [37]:
games = get_basic_boxscores(date='2023-03-01')
games['GAME_DATE'] = games['GAME_DATE'].dt.strftime('%Y-%m-%d')
games = games.sort_values('GAME_DATE', ascending=False)
betting_data = pd.read_pickle('../data/pkl/sbr_betting_data.pkl')
last_date = betting_data ['Game_Date'].iloc[0]
dates = games[games['GAME_DATE'] > last_date].sort_values('GAME_DATE', ascending=False)['GAME_DATE'].unique().tolist()


first = True
for date in dates:
    # time.sleep(4)
    URL = f"https://www.sportsbookreview.com/betting-odds/nba-basketball/?date={date}"
    headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"}

    page = requests.get(url=URL) #headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    teams = soup.find_all(class_='GameRows_participantBox__0WCRz')
    spread = soup.find_all(class_="OddsCells_compact__cawia border-left")
    wager_percentage = soup.find_all("span", class_="opener")
    scores = soup.find_all(class_ = 'GameRows_scores__YkN24')
    opening_spread = soup.find_all(class_='GameRows_adjust__NZn2m GameRows_opener__NivKJ')

    temp = []
    for j in range(int(7*len(teams)/2)):
        for w in [1, 2, 4, 5]:
            temp.append(spread[j].find_all("span")[w].text)

    spreads = []
    odds = []
    for i in range(len(temp)):
        if i % 2 == 0:
            spreads.append(temp[i])
        else:
            odds.append(temp[i])

    away = []
    home = []
    for i in range(len(spreads)):
        if i % 2 == 0:
            away.append(spreads[i])
            away.append(odds[i])
        else:
            home.append(spreads[i])
            home.append(odds[i])

    rows_to_add= []
    temp_away = []
    temp_home = []
    splits = [i for i in range(13, (14*20), 14)]
    for i in range(len(away)):
        temp_away.append(away[i])
        temp_home.append(home[i])
        if i in splits:
            rows_to_add.append(temp_away)
            rows_to_add.append(temp_home)
            temp_away = []
            temp_home = []

    row = []
    for i in range(len(teams)):
        if i % 2 == 0:
            opponent = teams[i+1].text
            home = 0
        else:
            opponent = teams[i-1].text
            home = 1
        row.append([date, teams[i].text, home, opponent, scores[i].text, wager_percentage[i].text, opening_spread[i].text, '-110'])

    df_1 = pd.DataFrame(row, columns=['Game_Date', 'Team_Name', 'Home', 'Opponent', 'Points', 'Pct_of_Bets', 'Opening_Spread', 'Opening_Odds'])

    columns = ['Betmgm_Spread', 'Betmgm_Odds', 'Draft_Kings_Spread', 'Draft_Kings_Odds',
            'Fanduel_Spread', 'Fanduel_Odds', 'Caesars_Spread', 'Caesars_Odds',
            'Pointsbet_Spread', 'Pointsbet_Odds', 'Wynn_Spread', 'Wynn_Odds',
            'Betrivers_Spread', 'Betrivers_Odds']

    df_2 = pd.DataFrame(rows_to_add, columns=columns)

    new_df = df_1.merge(df_2, left_index=True, right_index=True)

    if new_df.shape[0] != 0:
        if first == True:
            betting_df = new_df
            first = False
        else:
            betting_df = pd.concat([betting_df, new_df])

In [38]:
betting_df

,Game_Date,Team_Name,Home,Opponent,Points,Pct_of_Bets,Opening_Spread,Opening_Odds,Betmgm_Spread,Betmgm_Odds,...,Fanduel_Spread,Fanduel_Odds,Caesars_Spread,Caesars_Odds,Pointsbet_Spread,Pointsbet_Odds,Wynn_Spread,Wynn_Odds,Betrivers_Spread,Betrivers_Odds
0,2023-03-12,Brooklyn,0,Denver,122,55%,+8.5,-110,+9.5,-115,...,+9.5,-105,+9,-110,+9,-110,+9,-110,+9.5,-110
1,2023-03-12,Denver,1,Brooklyn,120,45%,-8.5,-110,-9.5,-105,...,-9.5,-114,-9,-110,-9,-110,-9,-110,-9.5,-117
2,2023-03-12,Cleveland,0,Charlotte,114,57%,-7.5,-110,-9.5,-115,...,-11.5,-108,-10,-110,-11.5,-105,-10,-110,-10,-110
3,2023-03-12,Charlotte,1,Cleveland,108,43%,+7.5,-110,+9.5,-105,...,+11.5,-120,+10,-110,+11.5,-120,+10,-110,+10,-113
4,2023-03-12,Washington,0,Philadelphia,93,44%,+8.5,-110,+7.5,-110,...,+7.5,-118,+7,-110,+7.5,-115,+7.5,-115,+8,-115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2023-03-06,Miami,1,Atlanta,130,68%,-3.5,-110,-2.5,-105,...,-2.5,-106,-2.5,-110,-2.5,-110,-2.5,-115,-3.5,-110
8,2023-03-06,Toronto,0,Denver,113,26%,+7.5,-110,+6.5,-110,...,+6.5,-112,+6.5,-110,+6,-110,+6,-110,+6,-109
9,2023-03-06,Denver,1,Toronto,118,74%,-7.5,-110,-6.5,-110,...,-6.5,-108,-6.5,-110,-6,-110,-6,-110,-6,-114
10,2023-03-06,New Orleans,0,Sacramento,108,28%,+6.5,-110,+4.5,-110,...,+4.5,-110,+4.5,-110,+4.5,-110,+4.5,-110,+4.5,-109


In [39]:
betting_data

,Game_Date,Team_Name,Home,Opponent,Points,Pct_of_Bets,Opening_Spread,Opening_Odds,Betmgm,Betmgm_Odds,...,Fanduel_Odds,Fanduel_Odds,Caesars_Odds,Caesars_Odds,Pointsbet_Odds,Pointsbet_Odds,Wynn_Odds,Wynn_Odds,Betrivers_Odds,Betrivers_Odds
0,2023-03-05,Phoenix,0,Dallas,130,56%,+1.5,-110,-1.5,-115,...,-2,-108,-2,-110,-2,-110,-2,-110,-2,-112
1,2023-03-05,Dallas,1,Phoenix,126,44%,-1.5,-110,+1.5,-105,...,+2,-112,+2,-110,+2,-110,+2,-110,+2,-112
2,2023-03-05,Indiana,0,Chicago,125,53%,+6.5,-110,+5.5,-110,...,+5.5,-108,+5,-110,+5.5,-110,+5.5,-110,+5.5,-110
3,2023-03-05,Chicago,1,Indiana,122,47%,-6.5,-110,-5.5,-110,...,-5.5,-112,-5,-110,-5.5,-110,-5.5,-110,-5.5,-112
4,2023-03-05,Golden State,0,L.A. Lakers,105,60%,-5.5,-110,-5.5,-105,...,-5.5,-118,-5.5,-110,-5,-110,-5,-110,-5,-109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2019-10-05,San Antonio,1,Orlando,89,-,-5.5,-110,,-,...,,-,,-,-6.5,-110,,-,,-
0,2019-10-04,Houston,0,L.A. Clippers,109,-,-4.5,-110,,-,...,,-,,-,-5,-110,,-,,-
1,2019-10-04,L.A. Clippers,1,Houston,96,-,+4.5,-110,,-,...,,-,,-,+5,-110,,-,,-
2,2019-10-04,Indiana,0,Sacramento,132,-,+2.5,-110,,-,...,,-,,-,+5.5,-110,,-,,-


In [45]:
pd.concat([betting_df, betting_data])

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [37]:
games = pd.read_pickle('../data/pkl/raw_games_5yrs.pkl')

games['GAME_DATE'] = games['GAME_DATE'].dt.strftime('%Y-%m-%d')

dates = games[games['GAME_DATE'] > '2019-10-01'].sort_values('GAME_DATE', ascending=False)['GAME_DATE'].unique().tolist()

In [17]:
date = '2020-07-08'

In [18]:
URL = f"https://www.sportsbookreview.com/betting-odds/nba-basketball/?date={date}"
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"}

In [19]:
page = requests.get(url=URL, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [20]:
teams = soup.find_all(class_='GameRows_participantBox__0WCRz')
spread = soup.find_all(class_="OddsCells_compact__cawia border-left")
wager_percentage = soup.find_all("span", class_="opener")
scores = soup.find_all(class_ = 'GameRows_scores__YkN24') 
opening_spread = soup.find_all(class_='GameRows_adjust__NZn2m GameRows_opener__NivKJ')

In [21]:
temp = []
for j in range(int(7*len(teams)/2)):
    for w in [1, 2, 4, 5]:
        temp.append(spread[j].find_all("span")[w].text)

In [22]:
spreads = []
odds = []
for i in range(len(temp)):
    if i % 2 == 0:
        spreads.append(temp[i])
    else:
        odds.append(temp[i])

In [23]:
away = []
home = []
for i in range(len(spreads)):
    if i % 2 == 0:
        away.append(spreads[i])
        away.append(odds[i])
    else:
        home.append(spreads[i])
        home.append(odds[i])

In [24]:
rows_to_add= []
temp_away = []
temp_home = []
splits = [i for i in range(13, (14*20), 14)]
for i in range(len(away)):
    temp_away.append(away[i])
    temp_home.append(home[i])
    if i in splits:
        rows_to_add.append(temp_away)
        rows_to_add.append(temp_home)
        temp_away = []
        temp_home = []

In [25]:
row = []
for i in range(len(teams)):
    if i % 2 == 0:
        opponent = teams[i+1].text
        home = 0
    else:
        opponent = teams[i-1].text
        home = 1
    row.append([date, teams[i].text, home, opponent, scores[i].text, wager_percentage[i].text, opening_spread[i].text, '-110'])

In [26]:
df_1 = pd.DataFrame(row, columns=['Game_Date', 'Team_Name', 'Home', 'Opponent', 'Points', 'Pct_of_Bets', 'Opening_Spread', 'Opening_Odds'])

In [27]:
columns = ['Betmgm', 'Betmgm_Odds', 'Draft_Kings_Odds', 'Draft_Kings_Odds', 
           'Fanduel_Odds', 'Fanduel_Odds', 'Caesars_Odds', 'Caesars_Odds', 
           'Pointsbet_Odds', 'Pointsbet_Odds', 'Wynn_Odds', 'Wynn_Odds', 
           'Betrivers_Odds', 'Betrivers_Odds']

In [28]:
df_2 = pd.DataFrame(rows_to_add, columns=columns)

In [29]:
betting_df = df_1.merge(df_2, left_index=True, right_index=True)
betting_df

,Game_Date,Team_Name,Home,Opponent,Points,Pct_of_Bets,Opening_Spread,Opening_Odds,Betmgm,Betmgm_Odds,...,Fanduel_Odds,Fanduel_Odds,Caesars_Odds,Caesars_Odds,Pointsbet_Odds,Pointsbet_Odds,Wynn_Odds,Wynn_Odds,Betrivers_Odds,Betrivers_Odds


In [35]:
if betting_df.shape[0] == 0:
    print('none')
else:
    print('not none')

none


In [34]:
betting_df.shape[0]

0